SAT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SAT version.

Necessary libraries:

In [40]:
#!pip3 install z3-solver

Necessary imports:

In [41]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [42]:
# open the file in Instances folder
f = open("Instances/inst14.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 20
items: 215
load_size: [200, 180, 160, 185, 180, 180, 200, 160, 180, 185, 200, 160, 175, 180, 160, 200, 180, 175, 160, 200]
item_size: [3, 16, 8, 15, 12, 13, 8, 24, 14, 17, 8, 11, 8, 17, 19, 13, 2, 8, 21, 13, 17, 4, 22, 24, 11, 6, 19, 11, 23, 4, 5, 16, 2, 13, 11, 9, 18, 7, 24, 5, 2, 5, 5, 3, 13, 2, 10, 12, 24, 21, 18, 5, 15, 11, 7, 3, 17, 14, 14, 7, 12, 5, 11, 17, 3, 7, 13, 20, 13, 4, 3, 24, 23, 21, 23, 10, 21, 24, 9, 22, 3, 6, 11, 10, 15, 9, 20, 19, 19, 19, 20, 1, 15, 18, 2, 25, 20, 18, 8, 3, 2, 15, 21, 6, 24, 19, 23, 16, 16, 9, 1, 10, 14, 11, 4, 23, 23, 22, 11, 14, 12, 13, 11, 9, 6, 25, 21, 13, 23, 8, 7, 8, 9, 2, 24, 7, 23, 21, 12, 23, 18, 5, 15, 11, 8, 2, 6, 23, 9, 1, 12, 17, 2, 5, 1, 20, 19, 13, 10, 9, 14, 4, 20, 17, 19, 19, 16, 4, 1, 21, 9, 2, 24, 1, 21, 13, 15, 4, 10, 19, 1, 25, 3, 21, 7, 18, 3, 1, 2, 8, 10, 19, 17, 19, 24, 24, 6, 2, 7, 10, 13, 20, 1, 17, 17, 16, 9, 25, 1, 23, 10, 15, 25, 25, 16]
distance:
 [[  0  81  75 ...  40 158 103]
 [ 81   0  19 ...  59  82  83]

Other variables:

In [43]:
COURIERS = np.arange(m)
ITEMS = np.arange(n)
items_total = sum(item_size)
courier_per_item = [Int("c_%s" % (i)) for i in ITEMS]

In [44]:
#courier_per_item

Formulation of the constraints:

In [45]:
constraints = []
# the constraint are the same as the CP model version 2
# number of items > number of couriers
constraints.append(n > m)

# Sum of item sizes cannot be bigger than total load size
constraints.append(items_total <= Sum([load_size[i] for i in COURIERS]))

# each item should have size > 0
for i in ITEMS:
    constraints.append(item_size[i] > 0)

# each courier should have load size > 0
for c in COURIERS:
    constraints.append(load_size[c] > 0)

# the couriers assigned should be between 0 and m-1
for i in ITEMS:
    constraints.append(And(courier_per_item[i] >= 0, courier_per_item[i] <= m-1))

# # Each courier delivers at least one item
# for c in COURIERS:
#     constraints.append(Sum([If(courier_per_item[i] == c, 1, 0) for i in ITEMS]) >= 1)
# ---------> this constraint is not necessary because of the next constraint <---------

# Each courier's items sizes must not exceed his load size and it should be > 0 as the courier can't be empty
load_per_courier = [Sum([If(courier_per_item[i] == c, item_size[i], 0) for i in ITEMS]) for c in COURIERS]
for c in COURIERS:
    constraints.append(And(load_per_courier[c] <= load_size[c], load_per_courier[c] > 0))
    
constraints
# print number of constraints
print(len(constraints))

472


In [46]:
#constraints

Solving the model:

In [47]:
# add all the constraints to the solver
s = Solver()
s.add(constraints)
#print("distance:\n", distance, "\n")
# solve
if s.check() == sat:
    print("     ---> sat <---")
    m = s.model()
    # print the model in a readable format
    print("c_i = courier of item i")
    # make a list of the couriers assigned to each item
    for c in COURIERS:
        print("Courier", c, ":", [i for i in ITEMS if m[courier_per_item[i]] == c])
    # commented below: calculate distance for each courier
    # for c in COURIERS:
    #     # list of items assigned to courier c
    #     items = [i for i in ITEMS if m[courier_per_item[i]] == c]
    #     distance_c = 0
    #     distance_c += distance[items[0]][n]
    #     for i in range(len(items)-1):
    #         distance_c += distance[items[i]][items[i+1]]
    #     distance_c += distance[items[-1]][n]
    #     print("Courier", c, "distance:", distance_c)
        
else:
    print("---> unsat <---")

     ---> sat <---
c_i = courier of item i
Courier 0 : [22, 59, 65, 66, 100, 110, 115, 127, 154, 155, 161, 163, 173, 176, 178, 193, 194]
Courier 1 : [7, 13, 27, 43, 64, 77, 83, 90, 114, 116, 124, 148, 180, 181]
Courier 2 : [21, 40, 63, 76, 85, 87, 96, 98, 99, 123, 145, 152, 187, 203, 207, 208]
Courier 3 : [1, 10, 15, 29, 39, 47, 53, 55, 57, 70, 71, 92, 94, 135, 150, 167, 172, 177, 188]
Courier 4 : [19, 28, 62, 67, 79, 84, 128, 132, 134, 157, 198]
Courier 5 : [5, 45, 48, 49, 56, 86, 142, 151, 168, 212, 213]
Courier 6 : [24, 26, 34, 58, 73, 97, 140, 141, 159, 175, 185, 195, 206, 210]
Courier 7 : [18, 102, 107, 108, 139, 171, 199, 201, 211, 214]
Courier 8 : [3, 17, 36, 89, 122, 137, 138, 164, 174, 179, 192]
Courier 9 : [4, 12, 42, 50, 75, 81, 95, 103, 118, 121, 133, 147, 160, 191, 200]
Courier 10 : [6, 11, 32, 35, 37, 68, 78, 91, 93, 104, 105, 131, 136, 146, 166, 184, 186, 190, 196]
Courier 11 : [2, 33, 41, 54, 74, 111, 113, 125, 126, 143, 149, 153, 158, 189]
Courier 12 : [8, 80, 169, 183

In [48]:
# # add all the constraints to the solver
# s = Solver()
# s.add(constraints)
# # solve
# if s.check() == sat:
#     m = s.model()